In [8]:
import pandas as pd
import os
os.getcwd()

'/home/smarda/BodyTweets/tweets_code'

In [35]:
un = pd.read_csv('../public_data/uncities.csv')
first_column = un.iloc[:,0]
len(first_column.unique())

3919

In [18]:
gb_df = pd.read_csv('../public_data/gbcities.csv')
gb_df.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,London,51.5072,-0.1275,United Kingdom,GB,"London, City of",primary,10979000,8825001
1,Birmingham,52.4800,-1.9025,United Kingdom,GB,Birmingham,NaN,2897303,1137100
2,Manchester,53.4794,-2.2453,United Kingdom,GB,Manchester,NaN,2705000,547627
3,Leeds,53.7997,-1.5492,United Kingdom,GB,Leeds,NaN,1901934,789194
4,Newcastle,55.0077,-1.6578,United Kingdom,GB,Newcastle upon Tyne,NaN,1650000,879996


In [16]:
canada_df = pd.read_csv('../public_data/canadacities.csv')
canada_df.head()

,city,city_ascii,province_id,province_name,lat,lng,population,density,timezone,ranking,postal,id
0,Toronto,Toronto,ON,Ontario,43.7417,-79.3733,5429524.0,4334.4,America/Toronto,1,M5T M5V M5P M5S M5R M5E M5G M5A M5C M5B M5M M5...,1124279679
1,Montréal,Montreal,QC,Quebec,45.5089,-73.5617,3519595.0,3889.0,America/Montreal,1,H1X H1Y H1Z H1P H1R H1S H1T H1V H1W H1H H1J H1...,1124586170
2,Vancouver,Vancouver,BC,British Columbia,49.2500,-123.1000,2264823.0,5492.6,America/Vancouver,1,V6Z V6S V6R V6P V6N V6M V6L V6K V6J V6H V6G V6...,1124825478
3,Calgary,Calgary,AB,Alberta,51.0500,-114.0667,1239220.0,1501.1,America/Edmonton,1,T1Y T2H T2K T2J T2L T2N T2A T2C T2B T2E T2G T2...,1124690423
4,Edmonton,Edmonton,AB,Alberta,53.5344,-113.4903,1062643.0,1360.9,America/Edmonton,1,T5X T5Y T5Z T5P T5R T5S T5T T5V T5W T5H T5J T5...,1124290735


In [27]:
foreign_entities = set(gb_df['city']).union(canada_df['city']).union(canada_df['province_name'])
{s.lower() for s in foreign_entities}

{'battle river no. 438',
 'cuckfield',
 'milton',
 'bracebridge',
 'new brunswick',
 'bilsthorpe',
 'saffron walden',
 'labelle',
 'sheringham',
 'sturminster newton',
 'cleator moor',
 'fort simpson',
 'wallace-woodworth',
 'aveley',
 'dinas powys',
 'marden',
 'cove',
 'ash vale',
 'dalton in furness',
 'kintore',
 'saint neots',
 'kidsgrove',
 'brislington',
 'skellingthorpe',
 'welshpool',
 'shippegan',
 'selston',
 'dymchurch',
 'saint-bruno-de-guigues',
 'tarporley',
 'bampton',
 'turner valley',
 'papineau-cameron',
 'hayle',
 'upham',
 'cap-chat',
 'capel saint mary',
 'macclesfield',
 'saint-gilles',
 'bromont',
 'topsham',
 'silkstone',
 'swillington',
 'godalming',
 'coppull',
 'dyce',
 'greenock',
 'coldstream',
 'st. andrews',
 'deanshanger',
 'bonavista',
 'meadow lake no. 588',
 'saint-alban',
 'sanderstead',
 'saint-esprit',
 'balderton',
 'selkirk',
 'appledore',
 'stratfield mortimer',
 'brixham',
 'lunenburg',
 'bicester',
 'alsager',
 'kintbury',
 'saint-éphrem-de-b

In [10]:
df = pd.read_csv('../public_data/usstateabbreviations.csv')
df.head()

,AK,Alaska
0,AL,Alabama
1,AR,Arkansas
2,AZ,Arizona
3,CA,California
4,CO,Colorado


In [ ]:
with open('')